In [2]:
from IPython.core.display import HTML

with open('style.html', 'r') as file:
    css = file.read()
HTML(css)

In [6]:
%run Util/00_imports.ipynb
%run Util/01_functions.ipynb

> TODO: Beschreibung der Variablen und deren Funktionsweise

In [7]:
STOCKFISH_PATH = "./stockfish/stockfish.exe"
VERBOSE = False

STOCKFISH = stockfish.Stockfish(STOCKFISH_PATH)

FileNotFoundError: [WinError 2] The system cannot find the file specified

> TODO: Beschreibung der Variablen

In [8]:
S_N_FILE = "S_n_seq_queen"

S_N_Sequence_fen, S_N_Sequence_fen_short = load_s_n_fens(S_N_FILE)

FileNotFoundError: [Errno 2] No such file or directory: 'S_n_Results/S_n_seq_queen.chessTest'

> TODO: Beschreibung der Funktion
> TODO: Kürzen der Funktionen -> Möglicherweise print() in separate Funktion auslagern oder so, stört beim Lesefluss
> TODO: anpassen mit indizes (wird dieser noch als Rückgabewert benötigt?

In [8]:
def find_next_move(fen, s_index, s_n_sequence_short):
    STOCKFISH.set_fen_position(fen)
    curr_board = chess.Board(fen)
    if curr_board.turn:
        if VERBOSE:
            print("---White:---")
            print("Starting in S" + str(s_index))
        for move in curr_board.legal_moves:
            curr_board.push(move)
            cur_fen = curr_board.fen()
            _tmp = find_situation_in_sequence(get_board_and_turn(cur_fen), [s_n_sequence_short[s_index - 1]])
            s_index_tmp = s_index - 1
            if _tmp != -1:
                if VERBOSE:
                    print("    Move: " + str(move))
                    print("    S" + str(s_index_tmp))
                    print("Ended in S" + str(s_index))
                curr_board.pop()
                return s_index_tmp, move
            curr_board.pop()

        return -1, None
    else:
        if VERBOSE:
            print("---Black:---")
            print("Starting in S" + str(s_index))
        move = chess.Move.from_uci(STOCKFISH.get_best_move())
        curr_board.push(move)
        cur_fen = curr_board.fen()
        s_index = find_situation_in_sequence(get_board_and_turn(cur_fen), s_n_sequence_short[:s_index])
        if VERBOSE:
            print("    Move: " + str(move))
            print("    S" + str(s_index))
            print("Ended in S" + str(s_index))
        curr_board.pop()
        return s_index, move

In [ ]:
# Split in white_turn und black_turn
def ki_white_move():
    pass

def ki_black_move():
    pass

> TODO: Beschreibung der Funktion

In [8]:
def calculate_all_moves(fen, s_n_sequence_short):
    moves = []

    s_index = find_situation_in_sequence(get_board_and_turn(fen), s_n_sequence_short)

    board = chess.Board(fen)
    while s_index > 0:
        cur_fen = board.fen()
        s_index, next_move = find_next_move(cur_fen, s_index, s_n_sequence_short)
        board.push(next_move)
        moves.append(next_move)

    if s_index == -1:
        return None

    return moves

> TODO: Beschreibung der Funktion
> TODO: Wird keine FEN mehr notwendig sein, lieber integer und diese in ein Board umwandeln

In [ ]:
def stockfish_movelist(fen):
    moves = []

    board = chess.Board(fen)

    while not board.is_game_over():
        STOCKFISH.set_fen_position(board.fen())
        next_move = chess.Move.from_uci(STOCKFISH.get_best_move())
        board.push(next_move)
        moves.append(next_move)

    return moves

> TODO: Beschreibung der Funktion
> TODO: Überlegung ob die nächsten drei (in dem Sinne statistischen) Funktionen wirklich notwendig sind; Hintergedanke, wenn Stockfish besser ist, ist die Frage nach dem warum
Kann man als potentielle Übersicht hinstellen, wobei das Betrachten im Fokus stehen sollte

In [8]:
def compare_move_lists(move_count_list):
    equal = 0
    ki_better = 0
    stockfish_better = 0
    for ki_move, stock_move, diff in move_count_list:
        if diff == 0:
            equal += 1
        elif diff < 0:
            ki_better += 1
        else:
            stockfish_better += 1
    return equal, ki_better, stockfish_better

> TODO: Beschreibung der Funktion


In [8]:
def get_average_difference(move_count_list):
    percentual_ki = []
    percentual_stock = []
    avg_ki_better = 0
    avg_stock_better = 0
    for ki_move, stock_move, diff in move_count_list:
        if diff < 0:
            percentual_ki.append(round(1 - (ki_move / stock_move), 4))
        elif diff > 0:
            percentual_stock.append(round(1 - (stock_move / ki_move), 4))
    if len(percentual_ki) != 0:
        avg_ki_better = sum(percentual_ki) / len(percentual_ki)
    if len(percentual_stock) != 0:
        avg_stock_better = sum(percentual_stock) / len(percentual_stock)
    return avg_ki_better, avg_stock_better

> TODO: Beschreibung der Funktion
> TODO: Wenn Statistiken nicht mehr benötigt werden, müssen Ergebnisse auch nicht mehr in Datei geschrieben werden

In [8]:
def write_result_to_file(filename, sequence_index, move_count_list):
    equal, ki_better, stockfish_better = compare_move_lists(move_count_list)
    avg_ki_better, avg_stock_better = get_average_difference(move_count_list)
    count = ki_better + stockfish_better + equal
    f = open("Tests/" + filename + ".txt", "a+")
    f.write("S_" + str(sequence_index) + ":\n")
    f.write("Stockfish war zu " + str(round((stockfish_better / count) * 100, 2)) + "% besser.\n")
    f.write("Die KI war zu " + str(round((ki_better / count) * 100, 2)) + "% besser.\n")
    f.write("Stockfish und die KI haben zu " + str(
        round((equal / count) * 100, 2)) + "% die gleichen Ergebnisse erzielt.\n")
    f.write("Sofern die KI besser war, hat sie durchschnittlich " + str(
        round(avg_ki_better * 100, 2)) + "% weniger Züge benötigt.\n")
    f.write("Sofern Stockfish besser war, hat sie durchschnittlich " + str(
        round(avg_stock_better * 100, 2)) + "% weniger Züge benötigt.\n")
    f.close()

# TESTS

> TODO: Beschreibung der Funktion
> TODO: Sollen alle Testergebnisse auch wirklich in einer Datei abgespeichert werden?
> TODO: Bei allen Tests eine Funktion, die den Vergleich von Stockfish und KI durchführt (ohne prints, nur Berechnung / Aufruf der bereits definierten Funktionen
> Wichtige Abklärung: Wenn sich herausstellt, dass Stockfish besser ist, wann soll die Anzeige passieren / Wie soll die Anzeige passieren? Sollen wir Konsole fluten mit einem / zwei Spielen
> Weitere Überlegung zum Vergleich: Die Move_History in dem Schachformat abspeichern und demnach abgleichen
> Bei Abgleich schauen, ob es an einem "falschen" Zug von weiß (also gewinnender Seite) ode rscharz liegt, weil wenn schwarz, dann liegt es an Stockfish und nicht der KI)

In [ ]:
def ki_vs_stockfish():
    pass

> TODO: Beschreibung der Funktion


In [8]:
def compare_fen_stockfish(fen, s_n_sequence_short):
    moves = calculate_all_moves(fen, s_n_sequence_short)
    stockfish_moves = stockfish_movelist(fen)

    if Moves is not None:
        print("AI needed " + str(len(moves)) + " moves to beat Stockfish as Black.")
    else:
        print("AI found no way to beat Black.")

    print("Stockfish needed " + str(len(stockfish_moves)) + " moves to win against itself.")

> TODO: Beschreibung der Funktion
> TODO: Bzgl. des Random Boards nochmal überlegen wie sinnig das ist ein Board aus den bisher klassifizierten auszuwählen

In [8]:
import random


def test_random_boards(count, s_n_sequence_fen, s_n_sequence_short):
    count_s_n = len(s_n_sequence_fen)
    move_count_list = []
    for board_c in range(count):
        rand_sequence = random.randint(0, count_s_n - 1)
        rand_board = random.randint(0, len(s_n_sequence_fen[rand_sequence]) - 1)
        rand_fen = list(s_n_sequence_fen[rand_sequence])[rand_board]
        ki_moves = calculate_all_moves(rand_fen, s_n_sequence_short)
        stockfish_moves = stockfish_movelist(rand_fen)
        move_count = len(ki_moves)
        cmp_move_count = len(stockfish_moves)
        move_count_list.append(tuple((move_count, cmp_move_count, move_count - cmp_move_count)))
        clear_output()
        print("Analyzed " + str(board_c + 1) + "/" + str(count))

    filename = "Random_" + str(count) + "_Compare_" + str(datetime.today().replace(microsecond=0)).replace(":",
                                                                                                           "_") + ".txt"
    write_result_to_file(filename, "random", move_count_list)

In [8]:
test_random_boards(100, S_N_Sequence_fen, S_N_Sequence_fen_short)

> TODO: Beschreibung der Funktion


In [8]:
def compare_sequence_stockfish(sequence_index, s_n_sequence_fen, s_n_sequence_short, g_filename=None):
    s_n = s_n_sequence_fen[sequence_index]
    move_count_list = []
    length = len(s_n)
    r = 0
    for board_fen in s_n:
        ai_moves = calculate_all_moves(board_fen, s_n_sequence_short)
        stockfish_moves = stockfish_movelist(board_fen)
        move_count = len(ai_moves)
        cmp_move_count = len(stockfish_moves)
        move_count_list.append(tuple((move_count, cmp_move_count, move_count - cmp_move_count)))
        clear_output()
        print("Comparing S_" + str(sequence_index) + ":")
        print("Compared " + str(r + 1) + "/" + str(length))
        r += 1

    if g_filename is None:
        filename = "S_" + str(sequence_index) + "_Compare_" + str(datetime.today().replace(microsecond=0)).replace(":",
                                                                                                                   "_") + ".txt"
        write_result_to_file(filename, sequence_index, move_count_list)
    else:
        write_result_to_file(g_filename, sequence_index, move_count_list)

In [8]:
compare_sequence_stockfish(20, S_N_Sequence_fen, S_N_Sequence_fen_short)

> TODO: Beschreibung der Funktion


In [8]:
def compare_all_sequences(s_n_sequence_fen, s_n_sequence_short):
    filename = "All_S_Compare_" + str(datetime.today().replace(microsecond=0)).replace(":", "_") + ".txt"
    for i in range(len(s_n_sequence_fen)):
        print("Comparing S_" + str(i) + "...")
        compare_sequence_stockfish(i, s_n_sequence_fen, s_n_sequence_short, filename)

In [8]:
compare_all_sequences(S_N_Sequence_fen, S_N_Sequence_fen_short)

> TODO: Beschreibung der Funktion
> TODO: Funktion nutzen, um Moves, wenn Stockfish besser ist anzuzeigen?
> TODO: Möglicherweise auch für einen Vergleich verwenden, wobei noch nicht ganz steht, wie der durchgeführt wird

In [8]:
def show_movelist(fen, moves):
    presentation_board = chess.Board(fen)
    display(presentation_board)
    time.sleep(2)
    for move in moves:
        presentation_board.push(move)
        clear_output(wait=True)
        display(presentation_board)
        time.sleep(2)
